In [1]:
import pandas as pd
from datetime import datetime as dt
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
print(f'Current working directory is {os.getcwd()}')

Current working directory is /Users/drewrichard/Documents/1 projects/nss/nss_projects/accre-carbonara/notebooks


In [21]:
# jobs = pd.read_csv("../data/complete.csv")
jobs = pd.read_csv("../data/fullsample.csv",
                    #nrows = 100000
                    )
ce5 = pd.read_csv('../data/ce5_unresponsive.csv')
ce6 = pd.read_csv('../data/ce6_unresponsive.csv')
jobs = jobs[jobs['END'] != 'Unknown']

In [22]:
jobs['END'] = pd.to_datetime(jobs['END'])
jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'])

In [23]:
len(jobs)

7395234

In [24]:
ce56 = pd.concat([ce5, ce6])
ce56['END'] = pd.to_datetime(ce56['END']).dt.floor('s')
len(ce56)

3296

In [25]:
unresponsive_merge = ce56.merge(jobs, left_on='END', right_on='END', how='left', validate='many_to_many')

In [31]:
unresponsive_merge = unresponsive_merge.dropna(subset='JOBID')

In [32]:
len(unresponsive_merge)

459

In [6]:
# jobs_ce = jobs.loc[(jobs['END'] >= '2020-10-17') & (jobs['END'] <= '2021-10-07')]
# ce56 = pd.concat([ce5, ce6])
# jobs_first = jobs_ce.loc[(jobs_ce['END'] >= '2020-10-18 06:16:25') & (jobs_ce['END'] <= '2020-10-18 06:38:44')]
# ce56_first = ce56.loc[(ce56['END'] >= '2020-10-18 00:00:00') & (ce56['END'] <= '2020-10-18 23:59:59')]
